In [1]:
import pandas as pd
import numpy as np
from statsmodels.stats.outliers_influence import variance_inflation_factor
import statsmodels.api as sm

In [2]:
df = pd.read_excel('3사분위수_데이터.xlsx')

In [6]:
myorigin = load_data('df')

print("\n===== 데이터 크기 확인 =====")
print(f"데이터셋 크기: {df.shape}")
print(f"열 개수: {df.shape[1]}")
print(f"행 개수: {df.shape[0]}")
print("\n===== 타입확인 =====")

print(df.info())

df.head()

df는 존재하지 않는 데이터에 대한 요청입니다.

===== 데이터 크기 확인 =====
데이터셋 크기: (13461, 11)
열 개수: 11
행 개수: 13461

===== 타입확인 =====
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13461 entries, 0 to 13460
Data columns (total 11 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 0  13461 non-null  int64  
 1   price       13461 non-null  int64  
 2   carat       13461 non-null  float64
 3   cut         13461 non-null  object 
 4   color       13461 non-null  object 
 5   clarity     13461 non-null  object 
 6   x           13461 non-null  float64
 7   y           13461 non-null  float64
 8   z           13461 non-null  float64
 9   depth       13461 non-null  float64
 10  table       13461 non-null  float64
dtypes: float64(6), int64(2), object(3)
memory usage: 1.1+ MB
None


,Unnamed: 0,price,carat,cut,color,clarity,x,y,z,depth,table
0,90,2757,0.70,Ideal,E,SI1,5.70,5.72,3.57,62.5,57.0
1,91,2757,0.86,Fair,E,SI2,6.45,6.33,3.52,55.1,69.0
2,92,2757,0.70,Ideal,G,VS2,5.70,5.67,3.50,61.6,56.0
3,93,2759,0.71,Very Good,E,VS2,5.68,5.73,3.56,62.4,57.0
4,94,2759,0.78,Very Good,G,SI2,5.81,5.85,3.72,63.8,56.0


In [8]:
df.shape

(13461, 11)

In [11]:
def ols_report(tbl, data):
    """
    OLS summary(tbl)와 데이터프레임(data)을 받아 회귀계수 표(rdf),
    모델 적합도 요약(result_report), 모형 보고 문장(model_report),
    변수별 보고 리스트(variable_reports)를 반환한다.
    Parameters
    ---------
    tbl : statsmodels.iolib.summary.Summary
        sm.OLS(...).fit().summary() 결과로 생성된 summary 객체.
    data : pandas.DataFrame
        종속변수와 독립변수를 모두 포함한 데이터프레임.
    Returns
    ------
    tuple
        (회귀계수 DataFrame, 적합도 요약 문자열, 모형 보고 문자열, 변수별 보고 리스트)
    """
    #독립변수 이름(상수항 제외)
    xnames = [n for n in fit.model.exog_names if n!="const"]

    #독립변수 부분 데이터(VIF 계산용)
    indi_df = data.filter(xnames)

    #독립변수 결과를 누적
    variables = []
    for i, v in enumerate(tbl.tables[1].data):
        #한 행의 변수명 추출 후 목록에 있는지 확인
        name = v[0].strip()
        if name not in xnames:
            continue
        #VIF계산을 위해 열 인덱스 조회
        j = list(indi_df.columns).index(name)
        vif = variance_inflation_factor(indi_df, j)

        #유의확률과 별표 표시 함수
        p = float(v[4].strip())
        stars = lambda p: (
            "***" if p < 0.001 else
            "**" if p < 0.01 else
            "*" if p < 0.05 else
            ""
        )

        #한 변수에 대한 보고 정보 추가
        variables.append({
            "종속변수": yname,
            "독립변수":name,
            "B":v[1].strip(),
            "표준오차":v[2].strip(),
            "Beta": float(fit.params[name]) * (data[name].std(ddof=1)/data[yname].std(ddof=1)), #표준화 회귀계수
            "t": "%s%s" % (v[3].strip(), stars(p)), #t-통계량(+별표)
            "p-value":p,
            "공차":1/vif,
            "vif":vif,
        })

    rdf = DataFrame(variables)

    # summary 표에서 적합도 정보를 key-value로 추출
    result_dict = {}
    for i in[0,2]:
        for item in tbl.tables[i].data:
            n = len(item)
            for i in range(0, n, 2):
                key = item[i].strip()[:-1]
                value = item[i+1].strip()
                if not key or not value:
                    continue
                result_dict[key] = value

    #적합도 보고 문자열 구성
    result_report =  f"𝑅({result_dict['R-squared']}), 𝑅^2({result_dict['Adj. R-squared']}), 𝐹({result_dict['F-statistic']}), 유의확률({result_dict['Prob (F-statistic)']}), Durbin-Watson({result_dict['Durbin-Watson']})"
    #모형 보고 문장 구성
    tpl = "%s에 대하여 %s로 예측하는 회귀분석을 실시한 결과, 이 회귀모형은 통계적으로 %s(F(%s,%s) = %s, p %s 0.05)."
    model_report = tpl % ( rdf['종속변수'][0],",".join(list(rdf['독립변수'])),"유의하다" if float(result_dict['Prob (F-statistic)']) <= 0.05 else "유의하지 않다",
        result_dict['Df Model'],
        result_dict['Df Residuals'],
        result_dict['F-statistic'],
        "<=" if float(result_dict['Prob (F-statistic)']) <= 0.05 else ">"
    )

    #변수별 보고 문장 리스트 구성
    variable_reports = []
    s = "%s의 회귀계수는 %s(p %s 0.05)로, %s에 대하여 %s 예측변인인 것으로 나타났다."

    for i in rdf.index:
        row = rdf.iloc[i]
        variable_reports.append(s % (row['독립변수'],row['B'],
                                     "<=" if float(row['p-value'])<0.05 else '>',
                                     row['종속변수'],
                                     '유의미한' if float(row['p-value'])<0.05 else '유의하지 않은' ))
        return rdf, result_report, model_report, variable_reports